# Manual CNV analysis

<b>Objective</b>

This vignette describes how copy number variation (CNV) can<br>
be identified using Mosaic. Data filtering, normalization and<br>
ploidy computation are described.

In [ ]:
import missionbio.mosaic as ms

sample = ms.load_example_dataset('3 cell mix')

This is an analyzed h5 file. Hence the clones and clusters<br>
are already labeled. It contains three cell lines KG-1,<br>
Tom-1, and Jurkat. It also contains doublets of each pair<br>
as seen in the heatmap.

For your h5 file, identify the clones using the DNA variants<br>
before proceeding with this vignette.

In [ ]:
fig = sample.dna.heatmap('NGT')
fig

### Filtering amplicons

We will only be working with the DNA and CNV assays.

The CNV object contains 138 amplicons. But not all<br>
of them worked as expected. We will drop amplicons<br>
which worked in less than half the total cells.

In [ ]:
# Remove amplicons with missing information.

# completeness : float [0, 100]
#     The minimum percentage of barcodes for an amplicon that must have
#     reads greater than or equal to `read_depth` to keep that amplicon.
# read_depth : int
#     The minimum required depth for each amplicon-barcode to not be
#     considered as missing during the completness calculation.

amplicons = sample.cnv.filter_amplicons(
    completeness=50,  # 50% of the cells
    read_depth=0,  # Should have greater than 0 reads
)

sample.cnv = sample.cnv[:, amplicons]

In [ ]:
# We are now left with 135 amplicons

sample.cnv.shape

### Normalization and ploidy computation

Finding the ploidy of the cells can be split into<br>
two distinct calculations.

* <b>Normalization</b>

    This step is required to correct for systemic artefacts.<br>
    Cell to cell, and amplicon to amplicon variations are corrected.<br>
    Each cell is normalized to it's total reads and each amplicon<br>
    to its median counts. This is performed using the `normalize_reads`<br>
    method, which adds the `normalized_counts` layer to the CNV object.


* <b>Correcting the baseline</b>

    Not all amplicons show copy loss or copy gain. In the normalization<br>
    step all amplicons in a cell were normalized using the same value<br>
    i.e. the total reads in the cell. This results in slight deviations<br>
    from the true ploidy value of the cell for each amplicon.
    
    To correct for this, a clone which is known to be diploid is identified<br>
    and all the other cells are normalized to the counts of this clone. This<br>
    step is done using the `compute_ploidy` method, and providing it<br>
    with the barcodes known to be diploid. This adds the `ploidy` layer<br>
    to the CNV object.

In [ ]:
# Reads are normalized to correct for systemic artefacts

sample.cnv.normalize_reads()

In [ ]:
# Assuming TOM-1 cells are diploid for all the amplicons,
# we can compute the ploidy of the other cell lines as follows

sample.cnv.compute_ploidy(diploid_cells=sample.dna.barcodes('TOM-1'))

### Visualizations

The ploidy values are stored in the `ploidy` layer

These can be visualized using a heatmap
or a ploidy line plot for each clone.

In [ ]:
# Assign the DNA labels to the CNV object
# We want to ensure the labels are the same

sample.cnv.set_labels(sample.dna)

#### Heatmap

In [ ]:
# Heatmap with the features ordered by the chromosome position

fig = sample.cnv.heatmap('ploidy', features='positions')
fig

In [ ]:
# Heatmap for a subset of the chromosomes

fig = sample.cnv.heatmap('ploidy', features=['4', '7', '20'])
fig

In [ ]:
# Heatmap with the features grouped by the genes
# The first time this runs, it fetches the gene names from ensembl
# The annotation can also be fetched using using sample.cnv.get_gene_names()

fig = sample.cnv.heatmap('ploidy', features='genes')
fig

In [ ]:
# Heatmap for a subset of the genes

fig = sample.cnv.heatmap('ploidy', features=['EZH2', 'TET2', 'TP53'])
fig

#### Line plot

As seen in the heatmap, KG-1 has a copy loss for some<br>
of the amplicons in the panel. TOM-1 does not show much<br>
deviation from ploidy 2.

This can also be visualized by the per amplicon ploidy plot.<br>
This plot is generated for each clone separately.

In [ ]:
# KG-1 shows copy loss at two different locations

fig = sample.cnv.plot_ploidy('KG-1')
fig

In [ ]:
# KG-1 shows copy loss at two different locations

fig = sample.cnv.plot_ploidy('KG-1')
fig

In [ ]:
# Jurkat is diploid for all the amplicons

fig = sample.cnv.plot_ploidy('Jurkat')
fig

In [ ]:
# Since TOM-1 was used to set the baseline,
# it is perfectly diploid for all amplicons

fig = sample.cnv.plot_ploidy('TOM-1')
fig